In [10]:
import pandas as pd

In [11]:
data = pd.read_csv('data/CombinedBinaryData.csv')
data

,QuestionText,QType,Answer,AnswerChoices,Year,Tags,Description
0,Will advanced LIGO announce discovery of gravi...,binary,no,NaN,2016,"['research impact', 'scientific discoveries', ...",NaN
1,Will SpaceX successfully land a Falcon 9 rocke...,binary,no,NaN,2016,"['app success', 'technological advances', 'phy...",NaN
2,Will we confirm evidence for megastructures or...,binary,no,NaN,2016,"['scientific discoveries', 'phys sci astro an...",NaN
3,Will Lockheed Martin and Skunkworks announce a...,binary,no,NaN,2019,"['research impact', 'technological advances', ...",NaN
4,The FAA to soon open the door to commercial us...,binary,no,NaN,2016,"['research impact', 'technological advances', ...",NaN
...,...,...,...,...,...,...,...
4458,"Between 3 October 2019 and 29 November 2019, w...",binary,no,NaN,2019,"['Idiosyncratic', 'Politics', 'International R...",The United Nations (UN) originally scheduled a...
4459,Will the PITF Worldwide Atrocities Dataset rec...,binary,yes,NaN,2019,"['PITF atrocities with non-state perpetrator',...","This question will resolve as ""yes"" if the Pol..."
4460,Will ACLED record any civilian fatalities in R...,binary,yes,NaN,2019,"['ACLED civilian fatalities', 'Politics', 'Int...",The Armed Conflict Location and Event Dataset ...
4461,Will ACLED record any civilian fatalities in S...,binary,yes,NaN,2019,"['ACLED civilian fatalities', 'Politics', 'Int...",The Armed Conflict Location and Event Dataset ...


In [20]:
startFrom, endAt = 4300, 4463

In [21]:
questions = list(data['QuestionText'])

In [22]:
from gpt4all import GPT4All
from tqdm import tqdm


def saveAnswers(questions, start, end):
    predictions = pd.DataFrame()
    predictions['predictions'] = questions
    predictions.to_csv(f'data/testSave{start}-{end}.csv')

model = GPT4All("wizardlm-13b-v1.2.Q4_0.gguf")
outputs = []
for i in tqdm(range(startFrom, endAt)):
    with model.chat_session():
        re = model.generate(f"### Instruction: \nAnswer the question below only with 'YES' or 'NO'. Feel free to hallucinate\n ### Question: \n{questions[i]} \n ### Response:\n", temp=0.5)
        # if the model said something that doesn't contain 'yes' or 'no', ask it to reanswer with 'yes' or 'no'
        if 'yes' not in re.lower() or 'no' not in re.lower():
            re = model.generate("### Instruction: \nOnly answer with 'YES' or 'NO' for the same question \n ###Response: \n")            
        outputs.append(re)
        if i % 100 == 0:
            saveAnswers(outputs, startFrom, i)

100%|██████████████████████████████████████████████████████████████████████████████| 163/163 [1:23:10<00:00, 30.62s/it]


In [23]:
predictions = pd.DataFrame()
predictions['predictions'] = outputs
predictions.to_csv(f'data/testSave4300-4462.csv')